### Create a dummy env_config

In [253]:
env_config = {}
env_config["curated_path"] = "/mnt/test/"
env_config["malcode"] = "caedw"

### Convert to Spark SQL

In [291]:
def convert_to_spark_sql(text,env_config):
    """[summary]

    Args:
        text ([type]): List genrated from synapse sql or hiveQL
        env_config ([type]): config object

    Returns:
        text1: list contains spark SQL 
    """
    text1 = []
    curated_path = env_config.get("curated_path")
    for elem in text:
        elem_converted = elem.replace("\n","").replace("\t","").strip().lower()
        if elem.lower().__contains__("create view"):
            elem_converted = elem_converted.replace("create view","create or replace view")
            elem_converted = elem_converted \
            .replace(elem_converted[(elem_converted.index("view")):int(elem_converted.index("as"))-1]
                    ,elem_converted[(elem_converted.index("view")):int(elem_converted.index("as"))-1]+"_vw") 
        if text.index(elem) != 0 and elem.lower().__contains__(env_config.get("malcode")):
            elem_converted = elem.replace("\n","").replace("\t","").strip().lower()
            start_index = elem_converted.index(env_config.get("malcode"))
            end_index = elem_converted.index(" ",elem_converted.index(env_config.get("malcode")))
            job_name_index = elem_converted.index(".",elem_converted.index(env_config.get("malcode")))
            job_name = elem_converted[job_name_index+1:end_index]        
            elem_converted = elem_converted.replace(elem_converted[start_index:end_index],f"delta.`{curated_path}{job_name}`")       
        text1.append(elem_converted)
    return text1

### Read from synapse SQL file, Convert to Spark SQL and Write to Output .sql file


In [293]:
with open('./data/sql2.sql','r') as file:
    text = file.read().lower()
text = text[text.index("create"):].split("\n")
#print(text)
text = convert_to_spark_sql(text,env_config)
#print(text)

f = open("./data/sql2_convrted.sql", "w")
for item in text:
   f.write(item + "\n")
f.close()

In [ ]:
# for elem in text:
#     if elem.__contains__("create"):
#         start_index = text.index(elem)
# text[start_index:] 

In [288]:
# text1 = []
# curated_path = env_config.get("curated_path")
# for elem in text:
#     elem_converted = elem.replace("\n","").replace("\t","").strip().lower()
#     if elem.lower().__contains__("create view"):
#         elem_converted = elem_converted.replace("create view","create or replace view")
#         elem_converted = elem_converted \
#         .replace(elem_converted[(elem_converted.index("view")):int(elem_converted.index("as"))-1]
#                  ,elem_converted[(elem_converted.index("view")):int(elem_converted.index("as"))-1]+"_vw") 
#     if text.index(elem) != 0 and elem.lower().__contains__(env_config.get("malcode")):
#         elem_converted = elem.replace("\n","").replace("\t","").strip().lower()
#         start_index = elem_converted.index(env_config.get("malcode"))
#         end_index = elem_converted.index(" ",elem_converted.index(env_config.get("malcode")))
#         job_name_index = elem_converted.index(".",elem_converted.index(env_config.get("malcode")))
#         job_name = elem_converted[job_name_index+1:end_index]        
#         elem_converted = elem_converted.replace(elem_converted[start_index:end_index],f"delta.`{curated_path}{job_name}`")       
#     text1.append(elem_converted)
# text1

### Converted SQL to output file

In [273]:
# f = open("./data/sql2_convrted.sql", "w")
# for item in text:
#    f.write(item + "\n")
# f.close()

In [ ]:
# lis1 = elem.split(".")
# print(lis1)

# for elem_view_name in lis1:
#     if elem_view_name.__contains__("as"):
#         print(str(elem_view_name[0:elem_view_name.index("as")-1])+"_vw")
# lis1